In [3]:
import pandas as pd
import numpy as np

In [9]:
from __future__ import print_function
from os.path import dirname
import os,sys,inspect

current_dir = dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
manage_dir = dirname(dirname(dirname(current_dir)))

if not manage_dir in sys.path:
    sys.path.insert(0, manage_dir)

# from pyml4.common import context, db
# context.local_context.print_summary()

import gensim
from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import logging
import nltk
import nltk.data
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from nltk.corpus import webtext
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.corpus import stopwords
import pymysql.cursors

In [7]:
def word2vec_synonyms(data):
    print("start finding synonyms using word2vec")
    tag_list = data['tag'].tolist()

    # load word2vedc model; edit the path and filename below;
    word_vec_model2 = Word2Vec.load('../models/Word2Vec_fullds_model2')

    # find synonyms using word2vec model;
    word2vec = []
    word2vec_2edits = []

    for index, row in data.iterrows():
        tag = row[0]
        temp = ''
        if tag in word_vec_model2.wv.vocab:
            for (k,v) in word_vec_model2.wv.similar_by_word(tag):
                if k in tag_list:
                    #temp.append(k)
                    temp += k+";"

        if len(temp) > 0:
            word2vec.append(temp)
        else:
            word2vec.append(None)

    # data = tag_no_cap_punc_stem.copy()
    new_data = data
    new_data['word2vec'] = word2vec
    return new_data

In [ ]:
def insert_to_sql(data_in_tuple):
    ### Connect and insert to SQL

    import pymysql.cursors
    connection = pymysql.connect(host="127.0.0.1",
                                 user="web",
                                 password="atth1132",
                                 db="ML3_dev")
    try:
        with connection.cursor() as cursor:
            # sql_drop = "DELETE from %s" % (tag_data)
#             sql_temp_table = """CREATE TABLE temp_tag_data (
#                                 tag varchar,
#                                 synonyms varchar,
#                                 tag_pref_candidates Boolean,
#                                 PRIMARY KEY (tag)
#                             );
#                             """
            sql_drop = "UPDATE tag_data SET synonyms = NULL;"
            cursor.execute(sql_drop)
            sql = """insert ignore into tag_data(tag, synonyms,tag_pref_candidates) 
                    VALUES(%s, %s, %s) 
                    ON DUPLICATE KEY UPDATE synonyms=VALUES(synonyms), tag_pref_candidates=VALUES(tag_pref_candidates)
                    """
            #for x in tuples:
            cursor.executemany(sql,data_in_tuple)
              #cursor.execute(sql,x)
#             sql_join_tables = """SELECT * FROM tag_data
#             LEFT JOIN 
#             """
            connection.commit()
    finally:
        connection.close()

#### load data

In [4]:
# This block counts how many times a tag has been applied; 
# Used for weighting user tags

import pymysql.cursors
import json

# Connect to the database
connection = pymysql.connect(host="127.0.0.1",
                             user="web",
                             password="atth1132",
                             db="ML3_dev")
try:
    with connection.cursor() as cursor:
            sql = "SELECT tag, score FROM tag_data"
            cursor.execute(sql)
            result = cursor.fetchall()
            
finally: 
    connection.close()




In [5]:
tag_data = pd.DataFrame(list(result), columns=['tag','score'])
#movie_tag_app.drop(["numPositive","numNeutral","numNegative","numDownvotes"],axis=1,inplace=True)

In [10]:
tag_data_synonyms = word2vec_synonyms(tag_data)

start finding synonyms using word2vec


/Users/evazhong/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [7]:
tag_pref_candidates = pd.read_csv("tag_genome_candidates_synonym_table.csv")

In [8]:
tag_pref_candidates = tag_pref_candidates["tag"].tolist()

In [16]:
tag_data_synonyms["tag_pref_candidates"] = False

In [19]:
tag_data_synonyms.loc[tag_data_synonyms["tag"].isin(tag_pref_candidates), "tag_pref_candidates"]=True

In [28]:
data_to_insert = tag_data_synonyms[["tag","word2vec","tag_pref_candidates"]]
tuples = [tuple(x) for x in data_to_insert.values]

[("!950's Superman TV show", None, False),
 ('!David O. Russell', None, False),
 ('!George Clooney', None, False),
 ('!George Lucas', None, False),
 ('"A Christmas Carol"', None, False),
 ('"A MÃ£o-de-Deus"', None, False),
 ('"American" French', None, False),
 ('"artsy"', None, False),
 ('"bad CIA"-- too simplistic', None, False),
 ('"based on true events"', None, False),
 ('"bitchy"', None, False),
 ('"damn dirty apes"', None, False),
 ('"Duck and cover!"', None, False),
 ('"eccentric genius"', None, False),
 ('"found footage"', None, False),
 ('"Ghost for adults"', None, False),
 ('"hero" is murderer', None, False),
 ('"I need a computer."', None, False),
 ('"inspired" by a book', None, False),
 ('"it wasn\'t expecially large, but it felt hard"', None, False),
 ('"It\'s not an American story', None, False),
 ('"Jack black"', None, False),
 ('"jackie chan" "chris tucker"', None, False),
 ('"Jessica Biel is hot like hell"', None, False),
 ('"magic negro" stereotype', None, False),
 ('"

In [57]:
insert_to_sql(tuples)

/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1364, "Field 'numApps' doesn't have a default value")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1364, "Field 'numPositive' doesn't have a default value")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1364, "Field 'numNeutral' doesn't have a default value")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1364, "Field 'numNegative' doesn't have a default value")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1364, "Field 'numDownvotes' doesn't have a default value")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1364, "Field 'score' doesn't have a 